In [ ]:
#This orders dams from upstream to downstream, assigning ToDam and FromDam fields

In [ ]:
#import packages

In [ ]:
#Load Data


### Remove duplicate dams on the same flowline.

In [ ]:
print('Size before duplicates removed:', noduplicates.shape)
NID_sort = noduplicates.sort_values('Hydroseq', ascending = True) # Sort by ascending Hydrosequence.

# Convert dataframe to dictionaries (struct-like); basically has format column->value.
NID_dict = NID_sort.to_dict(orient = 'records')
dupl_ordered_dict = NID.to_dict(orient='records')

# Initialize empty list to store indices of non-duplicates.
dupind = []

# Identify unique values and their counts.
Hydroseq = [item['Hydroseq'] for item in dupl_ordered_dict]
uniquevals,ia = np.unique(Hydroseq, return_inverse = True)

# Count the frequency of each index in ia.
bincounts = np.bincount(ia)

# Zero out singles.
singles = uniquevals[bincounts <= 1]
singleidx = [i for i, val in enumerate(Hydroseq) if val in singles]
for idx in singleidx:
    Hydroseq[idx] = 0
    
# Overwrite repeats.
repeats = uniquevals[bincounts > 1]
Hydroseq = np.array([np.where(repeats == val)[0][0] + 1 if val in repeats else val for val in Hydroseq])

#Set flags at duplicates as 5
flags[repeats] = [5]

In [ ]:
skip_it = 0; # Initialize a counter.

# Pull out fields needed to help with decision tree for removing duplicates.
YrC = [item['yrc'] for item in dupl_ordered_dict]
sites = [item['IsSite'] for item in dupl_ordered_dict]
grands = [item['IsGRanD'] for item in dupl_ordered_dict]
usbr = [item['IsUSBR'] for item in dupl_ordered_dict]
usace = [item['IsUSACE'] for item in dupl_ordered_dict]
MaxStor_m3 = [item['MaxStor_m3'] for item in dupl_ordered_dict]
DamH_m = [item['DamH_m'] for item in dupl_ordered_dict]
ShortID = [item['ShortID'] for item in dupl_ordered_dict]

for i in range(len(Hydroseq)):
    if Hydroseq[i] == 0: # If it is not a duplicate, keep it.
        dupind.append(i)
    elif skip_it > 0: # Or if we already dealt with it, update the counter so it gets skipped.
        skip_it -= 1
        continue
    else: # Else the value is a duplicate.
        dup = [idx for idx, val in enumerate(Hydroseq) if val == Hydroseq[i]] # Gives all indices of the duplicates.
        dup1 = dup[0]
        dupskip = dup1 # Keep track of what the first index was because we will change this.
        j = len(dup)
        jskip = j # Same for the length of the duplicates.

        MM = MaxStor_m3[dup1:dup[j-1]+1] # Pull out storage values at the duplicates.
        DamH = DamH_m[dup1:dup[j-1]+1] #Pull out dam heights at the duplicates
        SID = ShortID[dup1:dup[j-1]+1] #Pull out ShortIDs at the duplicates
#         kept_indices = [i for i, x in enumerate(Hydro) if x not in [Hydroseq[i] for i in dupind]] # If a dam is already snapped to that flowline, remove the flowlines from the options to choose from.

#         dup_test = dup
                
#         if len(dup_test) == 0: # All of the flowline options have already been used, in which case just keep them all. Duplicate snaps are removed later.
#             dup_test = dup
        
#         dup = dup_test
        
        dup1 = dup[0]
        j = len(dup)
        
        NoYr = [index for index, value in enumerate(YrC[dup1:dup[j-1]+1]) if value == 0] #pull out dams that don't have a year completed
        Year_Compl = [index for index, value in enumerate(YrC[dup1:dup[j-1]+1]) if value != 0] #pull out dams that do have a completion year
        siteloc = [index for index, value in enumerate(sites[dup1:dup[j-1]+1]) if value == 1] #sites
        GRanDloc = [index for index, value in enumerate(grands[dup1:dup[j-1]+1]) if value == 1] #GRanD dams
        USBRloc = [index for index, value in enumerate(usbr[dup1:dup[j-1]+1]) if value == 1] #USBR dams
        USACEloc = [index for index, value in enumerate(usace[dup1:dup[j-1]+1]) if value == 1] #USACE dams
        
        if len(siteloc) > 0: # If any of the dams are sites, choose that.
            dupind.append(dup[siteloc])
            continue #may not need these continues?
        elif len(GRanDloc) > 0: # If GRanD
            dup = dup[GRanDloc]
            
            if len(dup) > 1: #if for some reason multiple GRanD snapped to same flowline, keep both and print a warning
                dupind.append(dup)
                print('Error: Two GRanD dams snapped to the same flowline!')
            else:
                dupind.append(dup)
            continue
        elif len(USBRloc) > 0: #if usbr
            dupind.append(dup[USBRloc])
            continue
        elif len(USACEloc) > 0: #if usace
            dupin.append(dup[USACEloc])
            continue
        elif len(YearCompl) > 0: #if any of the dams have a completion year
            if len(YearCompl)<len(dup): #if all of the dams don't have a completion year, get rid of the ones that don't
                dup = np.delete(dup,NoYr) #delete any dams that don't have a year completed.
                MM = np.delete(MM,NoYr)
                HH = np.delete(HH,NoYr)
                SS = np.delete(SS,NoYr)
            
                maxloc = np.argmax(MM) #Take the maximum storage value
                dup = dup[maxloc]
            
                if len(dup)>1: #if more than one dam has the same storage value, take the largest dam height
                    HH = DamH[dup]
                    maxHH = np.argmax(HH)
                    dup = dup[maxHH]
                
                    if len(dup) > 1: #if the dam heights are the same, take the largest ShortID arbitrarily
                        SS = SID[dup]
                        maxSS = np.argmax(SS)
                        dup = dup[maxSS]
                dupind.append(dup)
                else: #otherwise just take the max storage
                    maxloc = np.argmax(MM) #Take the maximum storage value
                    dup = dup[maxloc]
            
                    if len(dup)>1: #if more than one dam has the same storage value, take the largest dam height
                        HH = DamH[dup]
                        maxHH = np.argmax(HH)
                        dup = dup[maxHH]
                
                        if len(dup) > 1: #if the dam heights are the same, take the largest ShortID arbitrarily
                            SS = SID[dup]
                            maxSS = np.argmax(SS)
                            dup = dup[maxSS]
                    dupind.append(dup)
                    continue
            else: #otherwise none of them have a year completed
                maxloc = np.argmax(MM) #Take the maximum storage value
                dup = dup[maxloc]
            
                if len(dup)>1: #if more than one dam has the same storage value, take the largest dam height
                    HH = DamH[dup]
                    maxHH = np.argmax(HH)
                    dup = dup[maxHH]
                
                    if len(dup) > 1: #if the dam heights are the same, take the largest ShortID arbitrarily
                        SS = SID[dup]
                        maxSS = np.argmax(SS)
                        dup = dup[maxSS]
                dupind.append(dup)
                
        if dupskip == i: # If the first index was the current index.
            skip_it = jskip-1 # Skip the next j-1 indices.
        else:
            skip_it = 0
                

flags = flags[dupind]
dupltable = pd.DataFrame.from_dict(dupl_ordered_dict)
noduplicates = dupltable.loc[dupind]

noduplicates.to_csv(os.path.join(out_folder,'NID_filtered_snapped_nodupl.csv'),index = False)


print('Size after removing duplicates:',noduplicates.shape) # New database size after snapping to NHD flowlines
print('Number of sites in database:', noduplicates.loc[noduplicates.IsSite == 1].shape)
print('Number of Reclamation dams in database:', noduplicates.loc[noduplicates.IsUSBR == 1].shape)
print('Number of USACE dams in database:', noduplicates.loc[noduplicates.IsUSACE == 1].shape)
print('Number of GRanD in database:', noduplicates.loc[noduplicates.IsGRanD == 1].shape)